In [1]:
from google.colab import drive
drive.mount('/content/drive')
!rm -rf /content/sample_dataxa

Mounted at /content/drive


In [2]:
import os
import json
import torch
from safetensors import safe_open
from transformers import DetrForObjectDetection, DetrConfig, DetrImageProcessor
from sklearn.metrics import roc_curve, auc, precision_recall_curve, f1_score
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt


In [3]:
!pip install timm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 59.0 MB/s eta 0:00:00


In [1]:
import os
import json
import torch
from safetensors import safe_open
from transformers import DetrForObjectDetection, DetrConfig, DetrImageProcessor

# Define the paths to the configuration and model weight files
config_path = '/content/config.json'  # Make sure this path is correct
model_weights_path = '/content/model.safetensors'  # Ensure this path is correct

# Load the configuration file
config = DetrConfig.from_json_file(config_path)

# Load the DETR model with the specified configuration
detr_model = DetrForObjectDetection(config)

# Load the weights from the safetensors file
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
state_dict = {}
with safe_open(model_weights_path, framework="pt") as f:
    for key in f.keys():
        state_dict[key] = f.get_tensor(key)

# Load the model state and move to the correct device
detr_model.load_state_dict(state_dict)
detr_model.to(device)
detr_model.eval()

# Initialize the image processor
detr_processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

In [1]:
import os
import torch
import torchvision.transforms as T
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone
from transformers import DetrForObjectDetection, DetrImageProcessor, DetrConfig
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from safetensors import safe_open

# Function to load the Faster R-CNN model with ResNet backbone
def load_faster_rcnn_model(model_path, backbone_name, num_classes):
    backbone = resnet_fpn_backbone(backbone_name, weights="DEFAULT")
    model = FasterRCNN(backbone, num_classes=num_classes)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.to(device)
    model.eval()
    return model

# Function to load the DETR model
def load_detr_model(config_path, model_weights_path):
    config = DetrConfig.from_json_file(config_path)
    model = DetrForObjectDetection(config)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    state_dict = {}
    with safe_open(model_weights_path, framework="pt") as f:
        for key in f.keys():
            state_dict[key] = f.get_tensor(key)
    model.load_state_dict(state_dict)
    model.to(device)
    model.eval()
    processor = DetrImageProcessor.from_pretrained("facebook/detr-resnet-50")
    return model, processor

# Function to get predictions from Faster R-CNN
def get_prediction(model, img_path, threshold=0.3):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    img = Image.open(img_path).convert("RGB")
    transform = T.Compose([T.ToTensor()])
    img = transform(img).to(device)
    model.eval()
    with torch.no_grad():
        prediction = model([img])
    pred_boxes = prediction[0]['boxes'].cpu().numpy()
    pred_scores = prediction[0]['scores'].cpu().numpy()
    pred_labels = prediction[0]['labels'].cpu().numpy()

    pred_t = [i for i, x in enumerate(pred_scores) if x >= threshold]

    if len(pred_t) == 0:
        return [], [], []

    pred_boxes = pred_boxes[pred_t]
    pred_scores = pred_scores[pred_t]
    pred_labels = pred_labels[pred_t]
    return pred_boxes, pred_labels, pred_scores

# Function to get predictions from DETR
def get_detr_prediction(model, processor, img_path, threshold=0.5):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    img = Image.open(img_path).convert("RGB")
    inputs = processor(images=img, return_tensors="pt").to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    target_sizes = torch.tensor([img.size[::-1]]).to(device)
    results = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=threshold)[0]
    pred_boxes = results["boxes"].cpu().numpy()
    pred_scores = results["scores"].cpu().numpy()
    pred_labels = results["labels"].cpu().numpy()
    return pred_boxes, pred_labels, pred_scores

# Function to save image with predictions
def save_image_with_boxes(img_path, boxes, labels, scores, color, model_name, save_path, threshold=0.5):
    img = Image.open(img_path).convert("RGB")
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(img)

    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            xmin, ymin, xmax, ymax = box
            width, height = xmax - xmin, ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
            plt.text(xmin, ymin - 10, f'{model_name}: Stenosis', fontsize=12, color='white',
                     bbox=dict(facecolor=color, edgecolor='none', alpha=0.7))

    ax.set_title(f'{model_name} Detection Results', fontsize=15, color='white')
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove any padding
    plt.savefig(save_path)
    plt.close()

# Load models
model_resnet50 = load_faster_rcnn_model('/content/drive/MyDrive/faster_rcnn_resnet50.pth', 'resnet50', 2)
model_resnet101 = load_faster_rcnn_model('/content/drive/MyDrive/faster_rcnn_resnet101.pth', 'resnet101', 2)
detr_model, detr_processor = load_detr_model('/content/config.json', '/content/model.safetensors')

# Example image
img_path = '/content/drive/MyDrive/po/test_images/14_021_34_0018.bmp'

# Get predictions from Faster R-CNN ResNet50 and save
pred_boxes_resnet50, pred_labels_resnet50, pred_scores_resnet50 = get_prediction(model_resnet50, img_path, threshold=0.5)
save_image_with_boxes(img_path, pred_boxes_resnet50, pred_labels_resnet50, pred_scores_resnet50, 'red', 'Faster R-CNN ResNet50', '/content/resnet50_predictions.png')

# Get predictions from Faster R-CNN ResNet101 and save
pred_boxes_resnet101, pred_labels_resnet101, pred_scores_resnet101 = get_prediction(model_resnet101, img_path, threshold=0.5)
save_image_with_boxes(img_path, pred_boxes_resnet101, pred_labels_resnet101, pred_scores_resnet101, 'blue', 'Faster R-CNN ResNet101', '/content/resnet101_predictions.png')

# Get predictions from DETR and save
pred_boxes_detr, pred_labels_detr, pred_scores_detr = get_detr_prediction(detr_model, detr_processor, img_path, threshold=0.5)
save_image_with_boxes(img_path, pred_boxes_detr, pred_labels_detr, pred_scores_detr, 'green', 'DETR', '/content/detr_predictions.png')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'backbone_name' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(
<ipython-input-1-d664dfe64838>:18: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=Tr

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/290 [00:00<?, ?B/s]

In [14]:
import os
import xml.etree.ElementTree as ET
from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

# Function to load the annotations from an XML file
def load_xml_annotations(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    annotations = []
    for obj in root.findall('object'):
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        annotations.append([xmin, ymin, xmax, ymax])
    return annotations

# Function to display image with annotations and save it
def display_and_save_image_with_annotations(img_path, annotations, save_path=None):
    img = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(img)

    for bbox in annotations:
        xmin, ymin, xmax, ymax = bbox
        draw.rectangle([xmin, ymin, xmax, ymax], outline="green", width=2)
        draw.text((xmin, ymin - 10), 'Ground Truth', fill="green")

    # Display the annotated image
    plt.figure(figsize=(10, 10))
    plt.imshow(img)
    plt.axis('off')

    if save_path:
        img.save(save_path)
        print(f"Annotated image saved to {save_path}")

    plt.show()

# Example paths
img_path = '/content/drive/MyDrive/po/test_images/14_021_34_0018.bmp'
annotations_file = '/content/drive/MyDrive/po/test_ann/14_021_34_0018.xml'

# Load annotations
annotations = load_xml_annotations(annotations_file)

# Save path for the annotated image
save_path = '/content/annotated_14_021_2_0026.png'

# Display the image with ground truth annotations and save it
display_and_save_image_with_annotations(img_path, annotations, save_path)


<IPython.core.display.Javascript object>

Annotated image saved to /content/annotated_14_021_2_0026.png


In [17]:
import torch
from pathlib import Path
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np

# Check if GPU is available and set the device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

# Load the YOLOv5 model and move to GPU
model = torch.hub.load('ultralytics/yolov5', 'custom', path='/content/best.pt', force_reload=True)
model = model.to(device)

# Image path to test
img_path = '/content/drive/MyDrive/po/test_images/14_021_34_0018.bmp'

# Function to load image
def load_image(img_path):
    img = Image.open(img_path).convert("RGB")
    return img

# Function to save image with predictions
def save_image_with_boxes(img_path, boxes, labels, scores, color, model_name, save_path, threshold=0.5):
    img = Image.open(img_path).convert("RGB")
    fig, ax = plt.subplots(1, figsize=(8, 8))
    ax.imshow(img)

    for box, label, score in zip(boxes, labels, scores):
        if score >= threshold:
            xmin, ymin, xmax, ymax = box
            width, height = xmax - xmin, ymax - ymin
            rect = patches.Rectangle((xmin, ymin), width, height, linewidth=2, edgecolor=color, facecolor='none')
            ax.add_patch(rect)
            plt.text(xmin, ymin - 10, f'{model_name}: Stenosis', fontsize=12, color='white',
                     bbox=dict(facecolor=color, edgecolor='none', alpha=0.7))

    ax.set_title(f'{model_name} Detection Results', fontsize=15, color='white')
    ax.axis('off')
    plt.subplots_adjust(left=0, right=1, top=1, bottom=0)  # Remove any padding
    plt.savefig(save_path)
    plt.close()

# Load the image
img = load_image(img_path)

# Run inference
results = model(img)

# Extract the predictions
boxes = results.xyxy[0][:, :4].cpu().numpy()  # Bounding boxes
scores = results.xyxy[0][:, 4].cpu().numpy()  # Confidence scores
labels = results.xyxy[0][:, 5].cpu().numpy()  # Class labels

# Define color and model name for the plot
color = 'red'
model_name = 'YOLOv5'

# Save and display the image with predictions
save_path = '/content/drive/MyDrive/predicted_image.png'
save_image_with_boxes(img_path, boxes, labels, scores, color, model_name, save_path)

print(f"Predicted image saved to {save_path}")

# Optionally, display the saved image
img_pred = Image.open(save_path)
plt.imshow(img_pred)
plt.axis('off')
plt.show()


Using device: cuda


Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-8-29 Python-3.10.12 torch-2.4.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
YOLOv5s summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Predicted image saved to /content/drive/MyDrive/predicted_image.png
